<a href="https://colab.research.google.com/github/YoungsikMoon/FORS/blob/main/%EB%AC%B8%EC%98%81%EC%8B%9D/FORS_%EB%AC%B8%EC%98%81%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCM 파일 핸들링 코드 모음

참고 : https://noggame.tistory.com/15

## pcm을 wav로 변환하는 코드

PCM 데이터를 WAV로 변환하려는 경우 WAV 파일이 어떻게 쓰였는지 나타내는 Header가 필요하다. Header에는 samplerate나 소리 데이터의 길이 등의 정보가 들어가며, 직접 구현하기보다는 numpy, librosa, soundfile 등의 라이브러리를 사용하면 간단히 해결 가능하다.

In [ ]:
import numpy as np
import librosa as lr
import soundfile as sf

target = "/content/KsponSpeech_000001.pcm" # 타깃 파일 경로
destinationPath = "/content/KsponSpeech_000001.wav" #만들 파일 경로
buf = None

with open(target, 'rb') as tf:
    buf = tf.read()
    buf = buf+b'0' if len(buf)%2 else buf    # padding 0 (경우에 따라서 PCM 파일의 길이가 8bit[1byte]로 나누어 떨어지지 않는 경우가 있어 0으로 패딩값을 더해준다, 해당 처리를 하지 않는 경우 numpy나 librosa 라이브러리 사용 시 오류가 날 수 있다)

pcm_data = np.frombuffer(buf, dtype='int16')
wav_data = lr.util.buf_to_float(x=pcm_data, n_bytes=2)
sf.write(destinationPath, wav_data, 16000, format='WAV', endian='LITTLE', subtype='PCM_16')

## wav 파일을 pcm 으로 변환하는 코드

PCM과 WAV의 소리 데이터 부분은 동일하고, Header 부분만 차이가 나므로 WAV 파일에서 (Header를 제외한) 데이터 부분만 읽어서 파일로 만들면 PCM 파일에 해당한다.

In [ ]:
target = "{대상파일_경로}"
destinationPath = target[:-4]+".pcm" # {생성파일경로}
buf = None

with open(destinationPath, "wb") as d_file:
    t_file = open(target, "rb")
    t_bin = t_file.read()
    d_file.write(t_bin[44:])    # header 이외의 데이터를 pcm 파일로 저장
    t_file.close()

## pcm 파일 통합 하는 코드

PCM은 파일의 시작에 소리 정보를 담고있는 header가 존재하지 않기 때문에, raw 데이터(Byte Code)를 그대로 읽어서 합치면된다.

In [ ]:
targetList = ["{대상파일1_경로}", "{대상파일2_경로}"]
destinationPath = "{생성파일경로}"

buf = bytearray()
for file in targetList:
    f = open(file, 'rb')
    buf += f.read()
    f.close()

wf = open(destinationPath, 'wb')
wf.write(buf)
wf.close()



---



# OpenAI WhisperAI STT (오래걸림)

위스퍼 설치

In [ ]:
pip install openai-whisper

한국어 모델 다운로드하기

In [ ]:
!whisper-cli --model ko --download-dir .

음성 파일 변환하기

변환할 한국어 음성 파일을 준비합니다.
다음 Python 코드를 실행하여 음성 파일을 텍스트로 변환합니다:

In [ ]:
import whisper

# 모델 로드
model = whisper.load_model("small")


결과 확인하기

위 코드를 실행하면 음성 파일의 텍스트 인식 결과가 출력됩니다.
인식 결과는 result['text']에 저장됩니다.

OpenAI Whisper는 다양한 언어를 지원하며, 특히 한국어 모델의 성능이 우수합니다. 이를 활용하면 다양한 음성 기반 애플리케이션을 개발할 수 있습니다.

추가로, Whisper는 다음과 같은 특징이 있습니다:

지원 언어: 총 98개 언어 지원 (한국어 포함)

모델 크기: 작은 모델부터 대형 모델까지 다양한 옵션 제공

사용 요금: 무료로 사용 가능

---



In [ ]:

# 음성 파일 경로 설정
audio_file = "/content/KsponSpeech_000001.wav"

# 음성 파일 변환
result = model.transcribe(audio_file)
print(f"인식 결과: {result['text']}")



---



# NVIDIA NeMo STT (1초)

참고 git : https://github.com/NVIDIA/NeMo

참고 문서 : https://huggingface.co/eesungkim/stt_kr_conformer_transducer_large

모델을 훈련하고, 미세 조정하고, 플레이하려면 NVIDIA NeMo를 설치해야 합니다 . 최신 Pytorch 버전을 설치한 후 설치하는 것이 좋습니다.

In [ ]:
!pip install nemo_toolkit['all']

이 모델은 NeMo 툴킷[1](https://github.com/NVIDIA/NeMo)에서 사용할 수 있으며 추론을 위해 사전 훈련된 체크포인트로 사용하거나 다른 데이터 세트에 대한 미세 조정을 수행할 수 있습니다.


자동으로 모델 인스턴스화

In [ ]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained("eesungkim/stt_kr_conformer_transducer_large")

음성파일 텍스트로 전환

In [19]:
asr_model.transcribe(['KsponSpeech_000001.wav'])

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

(['아 뭔 소리야 그건 또'], ['아 뭔 소리야 그건 또'])

많은 오디오 파일을 텍스트로 변환

In [ ]:
!python [NEMO_GIT_FOLDER]/examples/asr/transcribe_speech.py  pretrained_name="eesungkim/stt_kr_conformer_transducer_large"  audio_dir="<오디오 디렉터리>"

# 음성메모 프로그램 만들기

참고 : https://pagichacha.tistory.com/103

음성으로 말을하면 자동으로 기록해주는 메모 프로그램.

---



시나리오

1. 음성내용을 텍스트로  변환 후 파일 메모로 저장.
- 안내방송 (텍스트-음성 처리) 후 우리가 음성으로 말한 내용을 텍스ㅡ트로 변환 후 파일로 저장하는 방법에 대하여 살펴봅시다.

2. 위 1번 내용에서 음성이 끝나기 전까지 지속적으로 녹음되도록 개선
- 위의 1번의 경우 한번 말하면 끝이 나므로 다시 프로그램을 시작해야 하는 불편사항이 존재합니다. 따라서 우리가 특정 명령을 내리기 저까지는 계속 실행되어 메모를 진행할 수 있도록 대선하는 방법에 대하여 알아보겠습니다.



---



프로그램 실행하기 위한 모듈 임포트.

In [22]:
!pip install SpeechRecognition #음성인식

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 17.5 MB/s eta 0:00:00


In [24]:
!pip install gTTS #텍스트를 음성으로 만들 수 있다.

In [ ]:
!pip install pygame #음성 재생 기능

In [1]:
import speech_recognition as sr
from gtts import gTTS
import os
import time
import pygame
import os

In [ ]:


def speak(text):
    tts = gTTS(text=text, lang='ko')
    filename='voice.mp3'
    tts.save(filename)
    pygame.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        continue
    pygame.mixer.music.unload()
    time.sleep(1) # 1초 지연
    os.remove(filename)


def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("지금 말씀하세요: ")
        audio = r.listen(source)
        said = " "

        try:
            said = r.recognize_google(audio, language="ko-KR")
            print("말씀하신 내용입니다 : ", said)
        except Exception as e:
            print("Exception: " + str(e))

    return said

#############################
# 0.안내 방송(음성)
#############################
if os.path.isfile('memo.txt'):
    os.remove('memo.txt')
speak("안녕하세요. 2초 후에 말씀하세요. 종료시 굿바이 라고 말씀하시면 됩니다.")

while True:
#############################
# 1.음성입력
#############################
    text=get_audio()
    print(text)

#############################
# 5.파일저장
#############################
    with open('memo.txt', 'a') as f:
        f.write(str(text)+"\n")

    if "굿바이" in text:
        break

time.sleep(0.1)

# dataset 라이브러리의 load_datasets 함수 분석

FastWhisper 파인튜닝할 때 모질라의 commonvoice 라는 음성 코퍼스를 불러온다.
이 때 commonvoice를 load_datasets 함수로 불러오는데 어떻게 불러오는지 연구한다.
방법을 알면 우리의 음성코퍼스도 학습 시킬 수 있을것으로 생각된다.


In [ ]:
def load_dataset(
    path: str,
    name: Optional[str] = None,
    data_dir: Optional[str] = None,
    data_files: Optional[Union[str, Sequence[str], Mapping[str, Union[str, Sequence[str]]]]] = None,
    split: Optional[Union[str, Split]] = None,
    cache_dir: Optional[str] = None,
    features: Optional[Features] = None,
    download_config: Optional[DownloadConfig] = None,
    download_mode: Optional[Union[DownloadMode, str]] = None,
    verification_mode: Optional[Union[VerificationMode, str]] = None,
    ignore_verifications="deprecated",
    keep_in_memory: Optional[bool] = None,
    save_infos: bool = False,
    revision: Optional[Union[str, Version]] = None,
    token: Optional[Union[bool, str]] = None,
    use_auth_token="deprecated",
    task="deprecated",
    streaming: bool = False,
    num_proc: Optional[int] = None,
    storage_options: Optional[Dict] = None,
    trust_remote_code: bool = None,
    **config_kwargs,
) -> Union[DatasetDict, Dataset, IterableDatasetDict, IterableDataset]:
    """Load a dataset from the Hugging Face Hub, or a local dataset.

    You can find the list of datasets on the [Hub](https://huggingface.co/datasets) or with [`huggingface_hub.list_datasets`].

    A dataset is a directory that contains:

    - some data files in generic formats (JSON, CSV, Parquet, text, etc.).
    - and optionally a dataset script, if it requires some code to read the data files. This is used to load any kind of formats or structures.

    Note that dataset scripts can also download and read data files from anywhere - in case your data files already exist online.

    This function does the following under the hood:

        1. Download and import in the library the dataset script from `path` if it's not already cached inside the library.

            If the dataset has no dataset script, then a generic dataset script is imported instead (JSON, CSV, Parquet, text, etc.)

            Dataset scripts are small python scripts that define dataset builders. They define the citation, info and format of the dataset,
            contain the path or URL to the original data files and the code to load examples from the original data files.

            You can find the complete list of datasets in the Datasets [Hub](https://huggingface.co/datasets).

        2. Run the dataset script which will:

            * Download the dataset file from the original URL (see the script) if it's not already available locally or cached.
            * Process and cache the dataset in typed Arrow tables for caching.

                Arrow table are arbitrarily long, typed tables which can store nested objects and be mapped to numpy/pandas/python generic types.
                They can be directly accessed from disk, loaded in RAM or even streamed over the web.

        3. Return a dataset built from the requested splits in `split` (default: all).

    It also allows to load a dataset from a local directory or a dataset repository on the Hugging Face Hub without dataset script.
    In this case, it automatically loads all the data files from the directory or the dataset repository.

    Args:

        path (`str`):
            Path or name of the dataset.
            Depending on `path`, the dataset builder that is used comes from a generic dataset script (JSON, CSV, Parquet, text etc.) or from the dataset script (a python file) inside the dataset directory.

            For local datasets:

            - if `path` is a local directory (containing data files only)
              -> load a generic dataset builder (csv, json, text etc.) based on the content of the directory
              e.g. `'./path/to/directory/with/my/csv/data'`.
            - if `path` is a local dataset script or a directory containing a local dataset script (if the script has the same name as the directory)
              -> load the dataset builder from the dataset script
              e.g. `'./dataset/squad'` or `'./dataset/squad/squad.py'`.

            For datasets on the Hugging Face Hub (list all available datasets with [`huggingface_hub.list_datasets`])

            - if `path` is a dataset repository on the HF hub (containing data files only)
              -> load a generic dataset builder (csv, text etc.) based on the content of the repository
              e.g. `'username/dataset_name'`, a dataset repository on the HF hub containing your data files.
            - if `path` is a dataset repository on the HF hub with a dataset script (if the script has the same name as the directory)
              -> load the dataset builder from the dataset script in the dataset repository
              e.g. `glue`, `squad`, `'username/dataset_name'`, a dataset repository on the HF hub containing a dataset script `'dataset_name.py'`.

        name (`str`, *optional*):
            Defining the name of the dataset configuration.
        data_dir (`str`, *optional*):
            Defining the `data_dir` of the dataset configuration. If specified for the generic builders (csv, text etc.) or the Hub datasets and `data_files` is `None`,
            the behavior is equal to passing `os.path.join(data_dir, **)` as `data_files` to reference all the files in a directory.
        data_files (`str` or `Sequence` or `Mapping`, *optional*):
            Path(s) to source data file(s).
        split (`Split` or `str`):
            Which split of the data to load.
            If `None`, will return a `dict` with all splits (typically `datasets.Split.TRAIN` and `datasets.Split.TEST`).
            If given, will return a single Dataset.
            Splits can be combined and specified like in tensorflow-datasets.
        cache_dir (`str`, *optional*):
            Directory to read/write data. Defaults to `"~/.cache/huggingface/datasets"`.
        features (`Features`, *optional*):
            Set the features type to use for this dataset.
        download_config ([`DownloadConfig`], *optional*):
            Specific download configuration parameters.
        download_mode ([`DownloadMode`] or `str`, defaults to `REUSE_DATASET_IF_EXISTS`):
            Download/generate mode.
        verification_mode ([`VerificationMode`] or `str`, defaults to `BASIC_CHECKS`):
            Verification mode determining the checks to run on the downloaded/processed dataset information (checksums/size/splits/...).

            <Added version="2.9.1"/>
        ignore_verifications (`bool`, defaults to `False`):
            Ignore the verifications of the downloaded/processed dataset information (checksums/size/splits/...).

            <Deprecated version="2.9.1">

            `ignore_verifications` was deprecated in version 2.9.1 and will be removed in 3.0.0.
            Please use `verification_mode` instead.

            </Deprecated>
        keep_in_memory (`bool`, defaults to `None`):
            Whether to copy the dataset in-memory. If `None`, the dataset
            will not be copied in-memory unless explicitly enabled by setting `datasets.config.IN_MEMORY_MAX_SIZE` to
            nonzero. See more details in the [improve performance](../cache#improve-performance) section.
        save_infos (`bool`, defaults to `False`):
            Save the dataset information (checksums/size/splits/...).
        revision ([`Version`] or `str`, *optional*):
            Version of the dataset script to load.
            As datasets have their own git repository on the Datasets Hub, the default version "main" corresponds to their "main" branch.
            You can specify a different version than the default "main" by using a commit SHA or a git tag of the dataset repository.
        token (`str` or `bool`, *optional*):
            Optional string or boolean to use as Bearer token for remote files on the Datasets Hub.
            If `True`, or not specified, will get token from `"~/.huggingface"`.
        use_auth_token (`str` or `bool`, *optional*):
            Optional string or boolean to use as Bearer token for remote files on the Datasets Hub.
            If `True`, or not specified, will get token from `"~/.huggingface"`.

            <Deprecated version="2.14.0">

            `use_auth_token` was deprecated in favor of `token` in version 2.14.0 and will be removed in 3.0.0.

            </Deprecated>
        task (`str`):
            The task to prepare the dataset for during training and evaluation. Casts the dataset's [`Features`] to standardized column names and types as detailed in `datasets.tasks`.

            <Deprecated version="2.13.0">

            `task` was deprecated in version 2.13.0 and will be removed in 3.0.0.

            </Deprecated>
        streaming (`bool`, defaults to `False`):
            If set to `True`, don't download the data files. Instead, it streams the data progressively while
            iterating on the dataset. An [`IterableDataset`] or [`IterableDatasetDict`] is returned instead in this case.

            Note that streaming works for datasets that use data formats that support being iterated over like txt, csv, jsonl for example.
            Json files may be downloaded completely. Also streaming from remote zip or gzip files is supported but other compressed formats
            like rar and xz are not yet supported. The tgz format doesn't allow streaming.
        num_proc (`int`, *optional*, defaults to `None`):
            Number of processes when downloading and generating the dataset locally.
            Multiprocessing is disabled by default.

            <Added version="2.7.0"/>
        storage_options (`dict`, *optional*, defaults to `None`):
            **Experimental**. Key/value pairs to be passed on to the dataset file-system backend, if any.

            <Added version="2.11.0"/>
        trust_remote_code (`bool`, defaults to `True`):
            Whether or not to allow for datasets defined on the Hub using a dataset script. This option
            should only be set to `True` for repositories you trust and in which you have read the code, as it will
            execute code present on the Hub on your local machine.

            <Tip warning={true}>

            `trust_remote_code` will default to False in the next major release.

            </Tip>

            <Added version="2.16.0"/>
        **config_kwargs (additional keyword arguments):
            Keyword arguments to be passed to the `BuilderConfig`
            and used in the [`DatasetBuilder`].

    Returns:
        [`Dataset`] or [`DatasetDict`]:
        - if `split` is not `None`: the dataset requested,
        - if `split` is `None`, a [`~datasets.DatasetDict`] with each split.

        or [`IterableDataset`] or [`IterableDatasetDict`]: if `streaming=True`

        - if `split` is not `None`, the dataset is requested
        - if `split` is `None`, a [`~datasets.streaming.IterableDatasetDict`] with each split.

    Example:

    Load a dataset from the Hugging Face Hub:

    ```py
    >>> from datasets import load_dataset
    >>> ds = load_dataset('rotten_tomatoes', split='train')

    # Map data files to splits
    >>> data_files = {'train': 'train.csv', 'test': 'test.csv'}
    >>> ds = load_dataset('namespace/your_dataset_name', data_files=data_files)
    ```

    Load a local dataset:

    ```py
    # Load a CSV file
    >>> from datasets import load_dataset
    >>> ds = load_dataset('csv', data_files='path/to/local/my_dataset.csv')

    # Load a JSON file
    >>> from datasets import load_dataset
    >>> ds = load_dataset('json', data_files='path/to/local/my_dataset.json')

    # Load from a local loading script
    >>> from datasets import load_dataset
    >>> ds = load_dataset('path/to/local/loading_script/loading_script.py', split='train')
    ```

    Load an [`~datasets.IterableDataset`]:

    ```py
    >>> from datasets import load_dataset
    >>> ds = load_dataset('rotten_tomatoes', split='train', streaming=True)
    ```

    Load an image dataset with the `ImageFolder` dataset builder:

    ```py
    >>> from datasets import load_dataset
    >>> ds = load_dataset('imagefolder', data_dir='/path/to/images', split='train')
    ```
    """
    if use_auth_token != "deprecated":
        warnings.warn(
            "'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.\n"
            "You can remove this warning by passing 'token=<use_auth_token>' instead.",
            FutureWarning,
        )
        token = use_auth_token
    if ignore_verifications != "deprecated":
        verification_mode = VerificationMode.NO_CHECKS if ignore_verifications else VerificationMode.ALL_CHECKS
        warnings.warn(
            "'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.\n"
            f"You can remove this warning by passing 'verification_mode={verification_mode.value}' instead.",
            FutureWarning,
        )
    if task != "deprecated":
        warnings.warn(
            "'task' was deprecated in version 2.13.0 and will be removed in 3.0.0.\n",
            FutureWarning,
        )
    else:
        task = None
    if data_files is not None and not data_files:
        raise ValueError(f"Empty 'data_files': '{data_files}'. It should be either non-empty or None (default).")
    if Path(path, config.DATASET_STATE_JSON_FILENAME).exists():
        raise ValueError(
            "You are trying to load a dataset that was saved using `save_to_disk`. "
            "Please use `load_from_disk` instead."
        )

    if streaming and num_proc is not None:
        raise NotImplementedError(
            "Loading a streaming dataset in parallel with `num_proc` is not implemented. "
            "To parallelize streaming, you can wrap the dataset with a PyTorch DataLoader using `num_workers` > 1 instead."
        )

    download_mode = DownloadMode(download_mode or DownloadMode.REUSE_DATASET_IF_EXISTS)
    verification_mode = VerificationMode(
        (verification_mode or VerificationMode.BASIC_CHECKS) if not save_infos else VerificationMode.ALL_CHECKS
    )

    # Create a dataset builder
    builder_instance = load_dataset_builder(
        path=path,
        name=name,
        data_dir=data_dir,
        data_files=data_files,
        cache_dir=cache_dir,
        features=features,
        download_config=download_config,
        download_mode=download_mode,
        revision=revision,
        token=token,
        storage_options=storage_options,
        trust_remote_code=trust_remote_code,
        _require_default_config_name=name is None,
        **config_kwargs,
    )

    # Return iterable dataset in case of streaming
    if streaming:
        return builder_instance.as_streaming_dataset(split=split)

    # Some datasets are already processed on the HF google storage
    # Don't try downloading from Google storage for the packaged datasets as text, json, csv or pandas
    try_from_hf_gcs = path not in _PACKAGED_DATASETS_MODULES

    # Download and prepare data
    builder_instance.download_and_prepare(
        download_config=download_config,
        download_mode=download_mode,
        verification_mode=verification_mode,
        try_from_hf_gcs=try_from_hf_gcs,
        num_proc=num_proc,
        storage_options=storage_options,
    )

    # Build dataset for splits
    keep_in_memory = (
        keep_in_memory if keep_in_memory is not None else is_small_dataset(builder_instance.info.dataset_size)
    )
    ds = builder_instance.as_dataset(split=split, verification_mode=verification_mode, in_memory=keep_in_memory)
    # Rename and cast features to match task schema
    if task is not None:
        # To avoid issuing the same warning twice
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", FutureWarning)
            ds = ds.prepare_for_task(task)
    if save_infos:
        builder_instance._save_infos()

    return ds